In [148]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import beta
from scipy.stats import chi2
from scipy.stats import gamma
from scipy.stats import triang
from pulp import *

In [266]:
types = ['ball', 'bike', 'blocks', 'book', 'coal', 'doll', 'gloves', 'horse', 'train']
num_types = len(types)
key_to_item = dict((key, index) for key, index in zip(types, range(len(types))))
step = 0.1

In [23]:
def normal(x, mu, sigma):
    if x > 0:
        return norm.pdf(x, mu, sigma)
    return norm.cdf(0, mu, sigma)

weight_gen_map = {
    'horse': lambda: max(0, np.random.normal(5,2,1)[0]),
    'ball': lambda: max(0, 1 + np.random.normal(1,0.3,1)[0]),
    'bike': lambda: max(0, np.random.normal(20,10,1)[0]),
    'train': lambda: max(0, np.random.normal(10,5,1)[0]),
    'coal': lambda: 47 * np.random.beta(0.5,0.5,1)[0],
    'book': lambda: np.random.chisquare(2,1)[0],
    'doll': lambda: np.random.gamma(5,1,1)[0],
    'blocks': lambda: np.random.triangular(5,10,20,1)[0],
    'gloves': lambda: 3.0 + np.random.rand(1)[0] if np.random.rand(1) < 0.3 else np.random.rand(1)[0]
}

distribution_map = {
    'horse': lambda x: normal(x, *(5, 2)),
    'ball': lambda x: normal(x - 1, *(2, 0.3)),
    'bike': lambda x: normal(x, *(20,10)),
    'train': lambda x: normal(x, *(10,5)),
    'coal': lambda x: beta.pdf(x / 47 , 0.5, 0.5) / 47,
    'book': lambda x: chi2.pdf(x, 2),
    'doll': lambda x: (x**4) * np.exp(-x) / 25,
    'blocks': lambda x: triang.pdf(x, c = 1.0/3, loc = 5, scale = 15),
    'gloves': lambda x: 0.7 if (0 < x and x < 1) else 0.3 if (3 < x and x < 4) else 0
}

In [32]:
def save_expected_weights():
    expected_weight_list = [list(key) + [expected_weight_map[key]] for key in expected_weight_map]
    pd.DataFrame(expected_weight_list).to_csv('data/expected_weights', index=False)

def load_expected_weights():
    expected_weight_list = pd.read_csv('data/expected_weights')
    return dict([(tuple(int(val) for val in row[:9]), row[9]) for row in expected_weight_list.values])

In [5]:
def bag_to_key(bag):
    return tuple(bag)

def item_to_key(item):
    return tuple(1 if item == i else 0 for i in range(num_types))

def item_to_full_item(item):
    item_full = np.zeros((num_types,), dtype=np.int)
    item_full[item] += 1
    return item_full

In [6]:
def get_expected(bag):
    key = bag_to_key(bag)
    if key in expected_weight_map:
        return expected_weight_map[key]
    expected_weight = sum(prob * weight for prob, weight in zip(weight_map[key], weight_slices))
    expected_weight_map[key] = expected_weight
    return expected_weight

variance_reduce = 0.3
def get_expected_high_variance(bag):
    key = bag_to_key(bag)
    if key in expected_weight_map:
        return expected_weight_map[key]
    expected_weight = sum(prob * weight for prob, weight in zip(weight_map[key], weight_slices))
    chance_under = sum(weight_map[key])
    expected_weight /= (chance_under ** variance_reduce)
    expected_weight_map[key] = expected_weight
    return expected_weight

def get_chance_too_full(bag):
    key = bag_to_key(bag)
    if key in chance_too_full_map:
        return chance_too_full_map[key]
    chance_too_full = sum(weight_map[key])
    chance_too_full_map[key] = chance_too_full
    return 1 - chance_too_full

def calculate_combined_distribution(old_bag, item):
    bag = old_bag.copy()
    bag[item] += 1
    key = bag_to_key(bag)
    # already caculated
    if key in weight_map:
        return None
    
    distribution = np.zeros((500,), dtype=np.float)
    d1 = weight_map[bag_to_key(old_bag)]
    d2 = weight_array_singles[item]
    for p1, w1 in zip(d1, weight_indices):
        for p2, w2 in zip(d2, weight_indices):
            new_weight = w1 + w2
            if new_weight < 500:
                distribution[new_weight] += (p1 * p2)
    weight_map[key] = distribution
    return bag

# TODOS
- figure out how to do add_weights beter given that step/2 + step/2 isnt 3/2 step
- try increasing variance by increasing expected value if high chance of going over (multiply expected by (1/chance_under)^(0.5))
- iteratively try different subset swaps
- actually submit

In [7]:
# use this to populate map with combinations
# TODO make this faster and lower trial area
def hydrate_map(iterations=10):
    last_bags = [np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])]
    new_bags = []
    for i in range(iterations):
        for bag in last_bags:
            expected = get_expected(bag)
            for j in range(num_types):
                new_bag = calculate_combined_distribution(bag, j)
                if new_bag != None and get_expected(new_bag) > expected:
                    new_bags.append(new_bag)
        last_bags = new_bags
        new_bags = []

In [36]:
weight_map = {tuple([0] * num_types): np.zeros((500,), dtype=np.float)}
weight_map[tuple([0] * num_types)][0] = 1
expected_weight_map = {tuple([0] * num_types): 0}
chance_too_full_map = {tuple([0] * num_types): 0}

weight_slices = np.arange(step / 2, 50, step)
weight_indices = list(range(500))
weight_array_singles = [np.array([step * distribution_map[key](weight) for weight in weight_slices]) for key in types]
for weight_single in weight_array_singles:
    total_prob = sum(weight_single)
    for i in range(len(weight_single)):
        weight_single[i] /= total_prob

In [37]:
%%time
hydrate_map(50)
print (len(weight_map))
#save_expected_weights()

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


116168
CPU times: user 2h 34min 9s, sys: 33 s, total: 2h 34min 42s
Wall time: 2h 39min 39s


In [49]:
#expected_weight_map = load_expected_weights()

In [200]:
usefulness = [0.8, 0, 2.3, 2.0, 0, 1.05, 0.5, 1.2, 0.8]
useable_bags = [[np.array(key), expected_weight_map[key]] for key in expected_weight_map if sum(key) > 2]
sorted_normalized = sorted(useable_bags, key=lambda row: row[1] - sum(float(item) * factor for item, factor in zip(row[0], usefulness)))
sorted_normalized.reverse()

In [202]:
len(sorted_normalized)

116113

In [194]:
def solve_best_bags(useable_bags):
    counts = [1100, 500, 1000, 1200, 166, 1000, 200, 1000, 1000]
    bag_keys = LpVariable.dicts("bag", [str(i) for i in range(len(useable_bags))], 0, None, LpInteger)
    bag_names = [bag_keys[str(i)] for i in range(len(useable_bags))]
    prob = LpProblem("The Santa Uncertain Bags Problem", LpMaximize)

    # Add bag expected values
    prob += lpSum([bag[1] * bag_name for bag, bag_name in zip(useable_bags, bag_names)]), "objective"

    # Add item max constraints
    for count, i in zip(counts, range(len(counts))):
        prob += lpSum([bag[0][i] * bag_name for bag, bag_name in zip(useable_bags, bag_names)]) <= count, ""

    # Add bag maximum constraint    
    prob += lpSum(bag_names) <= 1000, ""

    prob.solve()
    print ("Status:", LpStatus[prob.status])
    print ("Score:", value(prob.objective))
    return [(useable_bags[int(var.name.split('_')[1])][0], int(var.varValue)) for var in prob.variables() if var.varValue != 0]

In [286]:
weight_gens = [
    lambda: max(0, 1 + np.random.normal(1,0.3,1)[0]),
    lambda: max(0, np.random.normal(20,10,1)[0]),
    lambda: np.random.triangular(5,10,20,1)[0],
    lambda: np.random.chisquare(2,1)[0],
    lambda: 47 * np.random.beta(0.5,0.5,1)[0],
    lambda: np.random.gamma(5,1,1)[0],
    lambda: 3.0 + np.random.rand(1)[0] if np.random.rand(1) < 0.3 else np.random.rand(1)[0],
    lambda: max(0, np.random.normal(5,2,1)[0]),
    lambda: max(0, np.random.normal(10,5,1)[0])
]
num_tests = 1000

In [310]:
def gen_weight(bag):
    weight = sum(weight_gen() for weight_gen, item_num in zip(weight_gens, bag) for j in range(item_num))
    return weight if weight <= 50 else 0

In [301]:
sorted_normalized[:10]

[[array([16,  0,  0,  0,  0,  0,  0,  0,  0]), 46.69832242138849],
 [array([15,  0,  0,  0,  0,  0,  1,  0,  0]), 45.195482550000605],
 [array([15,  0,  0,  0,  0,  0,  0,  0,  0]), 44.299976810829868],
 [array([13,  0,  0,  0,  0,  0,  0,  1,  0]), 43.297438176350681],
 [array([14,  0,  0,  0,  0,  0,  2,  0,  0]), 43.724964537990999],
 [array([13,  0,  0,  0,  0,  0,  1,  1,  0]), 43.360761076107039],
 [array([13,  0,  0,  0,  0,  1,  0,  0,  0]), 42.706284350339693],
 [array([14,  0,  0,  0,  0,  0,  0,  1,  0]), 43.600190198800917],
 [array([14,  0,  0,  0,  0,  0,  1,  0,  0]), 42.699949368059087],
 [array([11,  0,  0,  0,  0,  0,  0,  2,  0]), 42.180295486424562]]

In [ ]:
%%time
sorted_sampled = [(row[0], sum((gen_weight(row[0])) for i in range(num_tests)) / num_tests) for row in sorted_normalized[:1000]]

In [269]:
def score_bag(bag_counts):
    assert (sum(count for bag, count in bag_counts) == 1000)
    score_sum = 0
    for bag, count in bag_counts:
        for i in range(count):
            weight = sum(weight_gen_map[item_type]() for item_type, item_num in zip(types, bag) for j in range(item_num))
            if weight > 50:
                continue
            score_sum += weight
    return score_sum

In [212]:
bag_counts = solve_best_bags(sorted_normalized[:50000])

Status: Optimal
Score: 36110.36807102873


In [297]:
for i in range(10):
    print (score_bag(bag_counts))

35275.6782183
35058.9798178
34365.734187
35346.7707041
34481.6150485
35037.6340763
35109.0912403
35535.8879257
35195.609544
34395.5126702
